In [149]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import datetime
import statsmodels.tsa.api as smt
from IPython.display import display
from pandasql import *
from dateutil import parser
import pandasql as ps
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
  
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [150]:
def loadData(fileName):
    dir=os.path.dirname(os.path.abspath("__file__"))
    # dir = os.path.dirname(__file__)
    # print("abs---",dir)
    filename = dir+'\\data\\' + fileName;
    # print("filename--",filename)
    return pd.read_csv(filename, low_memory=False,encoding='latin-1')  

# Load flight Data

In [151]:
flight_data=loadData("selectedFlightData.csv")
# flight_data=loadData("Flight_Forecast_data.csv")

flight_data.head()



,Flight_No,org,dst,SCH_DEP_DATE,BKD_WT_sum,BKD_VOL_sum,CHG_WT_BU_sum,count,OPN_SUFX,INHOUSE_AC_TYP,...,AC_CGO_WT_BU,AC_CGO_VOL_BU,CMRCL_CGO_WT_BU,CMRCL_CGO_VOL_BU,NOOP_CAN_IND,ISO_CODE,EST_CGO_PLOD_WT_BU,EST_CGO_PLOD_VOL_BU,EST_WT_BU,EST_VOL_BU
0,6600,MUC,DWC,2018-01-19T00:00:00.000Z,13343.1,42.174486,13565.00,10,NaN,388L,...,15000.0,60.0,16000.0,60.0,NaN,GB,39915.06,90.498204,21662.10,27.677246
1,6600,MUC,DWC,2019-02-10T00:00:00.000Z,8771.4,69.009161,12379.77,18,NaN,388R,...,15000.0,60.0,15000.0,70.0,NaN,GB,25538.53,83.720000,5002.00,6.000000
2,6600,LIS,DWC,2018-07-24T00:00:00.000Z,14635.2,50.045059,15032.20,9,NaN,77WEJ,...,17000.0,90.0,17000.0,90.0,NaN,TR,16971.62,86.757365,707.84,0.885269
3,6600,LIS,DWC,2019-06-17T00:00:00.000Z,6509.6,37.940000,6509.60,6,NaN,77WER,...,20000.0,100.0,20000.0,100.0,NaN,TR,25584.45,98.077126,3139.68,3.950180
4,6600,LIS,DWC,2019-10-12T00:00:00.000Z,9936.0,52.440000,11709.34,5,NaN,77WER,...,20000.0,100.0,20000.0,100.0,NaN,TR,21075.26,84.050299,531.60,0.788024


In [152]:
flight_data['SCH_DEP_DATE'] = pd.to_datetime(flight_data['SCH_DEP_DATE']).dt.date

In [153]:
holiday_data=loadData("Holidays Final.csv")


In [154]:
airport_data=loadData("airport_code.csv")


In [155]:
def renameOrgDstFltCntryDfColumns(orgn_dst_flight_contry_df):
    orgn_dst_flight_contry_df['ORG_CNTRY_NAME'] = orgn_dst_flight_contry_df['ORG_CNTRY_NAME'].str.upper()
    orgn_dst_flight_contry_df['DST_CNTRY_NAME'] = orgn_dst_flight_contry_df['DST_CNTRY_NAME'].str.upper()
    orgn_dst_flight_contry_df['ORG_CNTRY_NAME'] = orgn_dst_flight_contry_df['ORG_CNTRY_NAME'].str.strip()
    orgn_dst_flight_contry_df['DST_CNTRY_NAME'] = orgn_dst_flight_contry_df['DST_CNTRY_NAME'].str.strip()

In [156]:
def getSelectedHolidayCol(selected_holiday_col_df):
    selected_holiday_col_df=selected_holiday_col_df[['Country',	'From_date','to_date']]
    selected_holiday_col_df['Country'] = selected_holiday_col_df['Country'].str.upper()
    selected_holiday_col_df['Country'] = selected_holiday_col_df['Country'].str.strip()
    selected_holiday_col_df['From_date'] = pd.to_datetime(selected_holiday_col_df['From_date']).dt.date
    selected_holiday_col_df['to_date'] = pd.to_datetime(selected_holiday_col_df['to_date']).dt.date

    selected_holiday_col_df=selected_holiday_col_df.assign  (countrylist=selected_holiday_col_df.Country.str.split(',')).explode('countrylist').reset_index(drop=True)
    # display(selected_holiday_col_df.shape[0])
    # res = selected_holiday_col_df.iloc[694:698,]
    # display(res)
    # print("holiday count--",len(selected_holiday_col_df.index))
    return selected_holiday_col_df

In [157]:
def getSelectedAirportCol(selected_airport_col_df):
    selected_airport_col_df=selected_airport_col_df[['AIRPORT_CODE','AIRPORT_NAME','COUNTRY_NAME',	'REGION_NAME']]
    return selected_airport_col_df

In [158]:
def getSelectedFlightCol(dataframe):
    selected_flights_colms_df=dataframe[['Flight_No','org','dst','SCH_DEP_DATE','BKD_WT_sum','BKD_VOL_sum','CHG_WT_BU_sum',
        'INHOUSE_AC_TYP','OPN_MODE','SVC_TYP','AC_CGO_WT_BU',	'AC_CGO_VOL_BU',
        'CMRCL_CGO_WT_BU','CMRCL_CGO_VOL_BU','NOOP_CAN_IND',	'ISO_CODE',	'EST_CGO_PLOD_WT_BU',
        'EST_CGO_PLOD_VOL_BU'
    ]]
    
    selected_flights_colms_df[selected_flights_colms_df["BKD_WT_sum"] < 0] = 0
    selected_flights_colms_df[selected_flights_colms_df["EST_CGO_PLOD_WT_BU"] < 0] = 0
    selected_flights_colms_df[selected_flights_colms_df["CHG_WT_BU_sum"] < 0] = 0
    
    return selected_flights_colms_df

In [159]:
selected_flight_col_df=getSelectedFlightCol(flight_data)
selected_holiday_col_df=getSelectedHolidayCol(holiday_data)
selected_aiport_col_df=getSelectedAirportCol(airport_data)

In [160]:
def writeToCsv(df,file_name):
    dir=os.path.dirname(os.path.abspath("__file__"))
    df.to_csv(dir+'\\data\\'+file_name)

# Join Origin of flight and country  

In [161]:

def joinOrgFlightAndContry(selected_flight_col_df,selected_aiport_col_df):
    
    flight_aiport_df=pd.merge(selected_flight_col_df,selected_aiport_col_df,how='left',left_on=['org'],right_on=        ['AIRPORT_CODE'])
    flight_aiport_df = flight_aiport_df.rename(columns={'COUNTRY_NAME': 'ORG_CNTRY_NAME',
                                'REGION_NAME':'ORG_REGION_NAME',
                                'AIRPORT_CODE':'ORG_AIRPORT_CODE',
                                'AIRPORT_NAME':'ORG_AIRPORT_NAME'})
    # display(flight_aiport_df)
    flight_aiport_df['ORG_CNTRY_NAME'] = flight_aiport_df['ORG_CNTRY_NAME'].str.upper()
    flight_aiport_df['ORG_CNTRY_NAME'] = flight_aiport_df['ORG_CNTRY_NAME'].str.strip()
    return flight_aiport_df
orgn_flight_contry_df=joinOrgFlightAndContry(selected_flight_col_df,selected_aiport_col_df)


# Join destination and country master

In [162]:
def joinDstFlightAndContry(selected_flight_col_df,selected_aiport_col_df):
    flight_aiport_df=pd.merge(selected_flight_col_df,selected_aiport_col_df,how='left',left_on=['dst'],right_on=['AIRPORT_CODE'])
    flight_aiport_df = flight_aiport_df.rename(columns={'COUNTRY_NAME': 'DST_CNTRY_NAME',
                                'REGION_NAME':'DST_REGION_NAME',
                                'AIRPORT_CODE':'DST_AIRPORT_CODE',
                                'AIRPORT_NAME':'DST_AIRPORT_NAME'})
    flight_aiport_df['DST_CNTRY_NAME'] = flight_aiport_df['DST_CNTRY_NAME'].str.upper()
    flight_aiport_df['DST_CNTRY_NAME'] = flight_aiport_df['DST_CNTRY_NAME'].str.strip()                        
    return flight_aiport_df;
orgn_dst_flight_contry_df=joinDstFlightAndContry(orgn_flight_contry_df,selected_aiport_col_df)

# Count and Replace NA

In [163]:
def replaceNAN(flt_airpot_holiday_df):
    nan_count=flt_airpot_holiday_df.isna().sum().sum()
    print("NAN count--",nan_count)
    flt_airpot_holiday_df = flt_airpot_holiday_df.replace(np.nan, 0)
    nan_count=flt_airpot_holiday_df.isna().sum().sum()
    print("NAN count--",nan_count)
    return flt_airpot_holiday_df;

# Find Origin holiday 

In [164]:

def joinFlighOrgAndHolidayData(orgn_dst_flight_contry_df,selected_holiday_col_df):
    flt_hlyday_flg_query = '''
        select flt.Flight_No,flt.org, flt.dst, flt.SCH_DEP_DATE, 1 as org_hday_flg
        from orgn_dst_flight_contry_df flt
        left join selected_holiday_col_df hldy 
        on (flt.ORG_CNTRY_NAME=hldy.Country or coalesce(hldy.Country,true))
        where flt.SCH_DEP_DATE >= hldy.From_date and flt.SCH_DEP_DATE <= hldy.to_date
        
    '''
    joined_flt_hlday_flag_df = ps.sqldf(flt_hlyday_flg_query,locals())

    final_flt_hlyday_query = '''
        select flt.*,coalesce(hldy.org_hday_flg,0) as org_hday_flg
        from orgn_dst_flight_contry_df flt
        left join joined_flt_hlday_flag_df hldy 
        on (flt.org=hldy.org 
            and flt.dst=hldy.dst 
            and flt.Flight_No=hldy.Flight_No 
            and flt.SCH_DEP_DATE=hldy.SCH_DEP_DATE
            )
    '''
    org_dst_flight_holidate_df = ps.sqldf(final_flt_hlyday_query,locals())

    return org_dst_flight_holidate_df

orgn_dst_flt_cntry_orgn_hlyday_df=joinFlighOrgAndHolidayData(orgn_dst_flight_contry_df,selected_holiday_col_df)
orgn_dst_flt_cntry_orgn_hlyday_df.head()
# display(orgn_dst_flt_cntry_orgn_hlyday_df)
# print("orgn holiday--",len(orgn_dst_flt_cntry_orgn_hlyday_df.index))
orgn_dst_flt_cntry_orgn_hlyday_df=orgn_dst_flt_cntry_orgn_hlyday_df.drop_duplicates()


In [165]:
def joinFlighDstAndHolidayData(orgn_dst_flight_contry_df,selected_holiday_col_df):
    # display(orgn_dst_flight_contry_df)
    # display(selected_holiday_col_df)
    
    flt_hlyday_flg_query = '''
        select flt.Flight_No,flt.org, flt.dst, flt.SCH_DEP_DATE, 1 as dst_hday_flg
        from orgn_dst_flight_contry_df flt
        left join selected_holiday_col_df hldy 
        on (flt.DST_CNTRY_NAME=hldy.Country or coalesce(hldy.Country,true) )
        where flt.SCH_DEP_DATE >= hldy.From_date and flt.SCH_DEP_DATE <= hldy.to_date
    '''
    joined_flt_hlday_flag_df = ps.sqldf(flt_hlyday_flg_query,locals())

    final_flt_hlyday_query = '''
        select flt.*, coalesce(hldy.dst_hday_flg,0) as dst_hday_flg
        from orgn_dst_flight_contry_df flt
        left join joined_flt_hlday_flag_df hldy 
        on (flt.org=hldy.org 
            and flt.dst=hldy.dst 
            and flt.Flight_No=hldy.Flight_No 
            and flt.SCH_DEP_DATE=hldy.SCH_DEP_DATE
            )
    '''
    org_dst_flight_holidate_df = ps.sqldf(final_flt_hlyday_query,locals())
    return org_dst_flight_holidate_df

orgn_dst_flt_cntry_dstnorgn_hlyday_df=joinFlighDstAndHolidayData(orgn_dst_flt_cntry_orgn_hlyday_df,selected_holiday_col_df)
orgn_dst_flt_cntry_dstnorgn_hlyday_df.head()

,Flight_No,org,dst,SCH_DEP_DATE,BKD_WT_sum,BKD_VOL_sum,CHG_WT_BU_sum,INHOUSE_AC_TYP,OPN_MODE,SVC_TYP,...,ORG_AIRPORT_CODE,ORG_AIRPORT_NAME,ORG_CNTRY_NAME,ORG_REGION_NAME,DST_AIRPORT_CODE,DST_AIRPORT_NAME,DST_CNTRY_NAME,DST_REGION_NAME,org_hday_flg,dst_hday_flg
0,6600,MUC,DWC,2018-01-19,13343.1,42.174486,13565.00,388L,J,J,...,MUC,International,GERMANY,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0
1,6600,MUC,DWC,2019-02-10,8771.4,69.009161,12379.77,388R,J,J,...,MUC,International,GERMANY,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0
2,6600,LIS,DWC,2018-07-24,14635.2,50.045059,15032.20,77WEJ,J,J,...,LIS,Airport,PORTUGAL,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0
3,6600,LIS,DWC,2019-06-17,6509.6,37.940000,6509.60,77WER,J,J,...,LIS,Airport,PORTUGAL,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0
4,6600,LIS,DWC,2019-10-12,9936.0,52.440000,11709.34,77WER,J,J,...,LIS,Airport,PORTUGAL,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,1


# Drop Columns

In [166]:
def dropColumns(flt_airpot_holiday_df):
    colm_array=['ORG_CNTRY_HOLIDAY','ORG_HOLIDAY_FROM_DATE','ORG_HOLIDAY_TO_DATE','DST_CNTRY_HOLIDAY',	    'DST_HOLIDAY_FROM_DATE','DST_HOLIDAY_TO_DATE','ORG_REGION_NAME','DST_REGION_NAME','ORG_AIRPORT_CODE',	'ORG_AIRPORT_NAME','ORG_CNTRY_NAME','DST_AIRPORT_CODE','DST_AIRPORT_NAME','DST_CNTRY_NAME']
    flt_airpot_holiday_df_droped=flt_airpot_holiday_df.drop(colm_array, axis = 1)
    return flt_airpot_holiday_df_droped;

In [167]:
# flt_airpot_holiday_drop_df=dropColumns(flt_airpot_holiday_df)
flt_airpot_holiday_drop_df=replaceNAN(orgn_dst_flt_cntry_dstnorgn_hlyday_df)
flt_airpot_holiday_drop_dup=flt_airpot_holiday_drop_df.drop_duplicates()
# display(flt_airpot_holiday_drop_dup)
flt_airpot_holiday_drop_dup.head()



NAN count-- 12301
NAN count-- 0


,Flight_No,org,dst,SCH_DEP_DATE,BKD_WT_sum,BKD_VOL_sum,CHG_WT_BU_sum,INHOUSE_AC_TYP,OPN_MODE,SVC_TYP,...,ORG_AIRPORT_CODE,ORG_AIRPORT_NAME,ORG_CNTRY_NAME,ORG_REGION_NAME,DST_AIRPORT_CODE,DST_AIRPORT_NAME,DST_CNTRY_NAME,DST_REGION_NAME,org_hday_flg,dst_hday_flg
0,6600,MUC,DWC,2018-01-19,13343.1,42.174486,13565.00,388L,J,J,...,MUC,International,GERMANY,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0
1,6600,MUC,DWC,2019-02-10,8771.4,69.009161,12379.77,388R,J,J,...,MUC,International,GERMANY,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0
2,6600,LIS,DWC,2018-07-24,14635.2,50.045059,15032.20,77WEJ,J,J,...,LIS,Airport,PORTUGAL,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0
3,6600,LIS,DWC,2019-06-17,6509.6,37.940000,6509.60,77WER,J,J,...,LIS,Airport,PORTUGAL,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0
4,6600,LIS,DWC,2019-10-12,9936.0,52.440000,11709.34,77WER,J,J,...,LIS,Airport,PORTUGAL,EUROPE,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,1


In [168]:
# Calculate Chargeable and Booked weight Ratio
def calEstimatedChgWeight(orgn_dst_flight_contry_df):
    orgn_dst_flight_contry_df.loc[(orgn_dst_flight_contry_df['CHG_WT_BU_sum'] <0)] = 1 
    orgn_dst_flight_contry_df.loc[(orgn_dst_flight_contry_df['BKD_WT_sum'] <0)] = 1 
    grp_org_dst_wt_query = '''
        select flt.org,flt.dst,sum(flt.CHG_WT_BU_sum)/sum(flt.BKD_WT_sum) as CHG_WT_RT
        from orgn_dst_flight_contry_df flt
        group by flt.org, flt.dst
    '''
    grpd_orgdst_chrgbl_rt_df = ps.sqldf(grp_org_dst_wt_query,locals())
    grpd_orgdst_chrgbl_rt_df = grpd_orgdst_chrgbl_rt_df.replace(np.nan, 0)
    # grpd_orgdst_chrgbl_rt_df[CHG_WT_RT < 0] = 0
    display(grpd_orgdst_chrgbl_rt_df)
    grpd_orgdst_chrgbl_rt_df.loc[(grpd_orgdst_chrgbl_rt_df['CHG_WT_RT'] <0)] = 1 
    chrbl_cal_query = '''
        select flt.*,(flt.EST_CGO_PLOD_WT_BU * rt.CHG_WT_RT) as EST_CHG_WT
        from orgn_dst_flight_contry_df flt
        left join grpd_orgdst_chrgbl_rt_df rt 
        on (flt.org=rt.org 
            and flt.dst=rt.dst 
            )
        where flt.dst=rt.dst 
    '''
    grpd_orgdst_chrgbl_rt_df = ps.sqldf(chrbl_cal_query,locals())

    # display(grpd_orgdst_chrgbl_rt_df)
    return grpd_orgdst_chrgbl_rt_df


grpd_orgdst_chrgbl_rt_df=calEstimatedChgWeight(flt_airpot_holiday_drop_dup);


print("final df")
# display(grpd_orgdst_chrgbl_rt_df)
grpd_orgdst_chrgbl_rt_df.head()

slct_df = grpd_orgdst_chrgbl_rt_df[(grpd_orgdst_chrgbl_rt_df.org == 'DWC') &(grpd_orgdst_chrgbl_rt_df.dst == 'ICN')]
grpd_orgdst_chrgbl_rt_df=slct_df.dropna()
grpd_orgdst_chrgbl_rt_df

writeToCsv(grpd_orgdst_chrgbl_rt_df,"final_data.csv")


,org,dst,CHG_WT_RT
0,0,0,0.000000
1,ADD,DWC,1.026262
2,AKL,SIN,1.053646
3,AKL,SYD,1.588971
4,BNE,SIN,1.111133
5,DME,DWC,1.023752
6,DMM,DWC,1.404216
7,DWC,AKL,1.193209
8,DWC,BAH,1.213317
9,DWC,BNE,1.006822


final df


In [143]:

stepwise_fit = auto_arima(grpd_orgdst_chrgbl_rt_df['CHG_WT_BU_sum'], start_p = 1, start_q = 1,
                          max_p = 3, max_q = 3, m = 12,
                          start_P = 0, seasonal = True,
                          d = None, D = 1, trace = True,
                          error_action ='ignore',   # we don't want to know if an order does not work
                          suppress_warnings = True,  # we don't want convergence warnings
                          stepwise = True)           # set to stepwise
  
# To print the summary
stepwise_fit.summary()

# train = airline.iloc[:len(airline)-12]
# test = airline.iloc[len(airline)-12:] # set one year(12 months) for testing
  
# # Fit a SARIMAX(0, 1, 1)x(2, 1, 1, 12) on the training set
# from statsmodels.tsa.statespace.sarimax import SARIMAX
  
# model = SARIMAX(train['# Passengers'], 
#                 order = (0, 1, 1), 
#                 seasonal_order =(2, 1, 1, 12))
  
# result = model.fit()
# result.summary()



In [144]:
# mask = grpd_orgdst_chrgbl_rt_df['org']=='DWC' and grpd_orgdst_chrgbl_rt_df['dst']=="ICN"
# slct_df=grpd_orgdst_chrgbl_rt_df.where(mask)


In [145]:
featurs=['SCH_DEP_DATE']

In [146]:
def getDailyCargoData(data):
    daily_data = data.copy()
    # display(daily_data)
    # daily_data.SCH_DEP_DATE = daily_data.SCH_DEP_DATE.apply(lambda x: str(x)[:-3])
    daily_data = daily_data.groupby("SCH_DEP_DATE")['CHG_WT_BU_sum'].sum().reset_index()
    # daily_data["SCH_DEP_DATE"] = pd.to_datetime(daily_data.SCH_DEP_DATE)
    flight_data.SCH_DEP_DATE = pd.to_datetime(flight_data.SCH_DEP_DATE, format='%Y-%m-%d')
    flight_data['SCH_DEP_DATE'] = pd.to_datetime(flight_data['SCH_DEP_DATE']).dt.date
    display(daily_data)
    return daily_data

daily_data=getDailyCargoData(grpd_orgdst_chrgbl_rt_df)
daily_data=daily_data.sort_values(by='SCH_DEP_DATE')


,SCH_DEP_DATE,CHG_WT_BU_sum
0,2018-01-03,171.1
1,2018-01-04,815.2
2,2018-01-05,551.7
3,2018-01-06,1820.2
4,2018-01-07,12000.3
...,...,...
1178,2021-09-14,1000.0
1179,2021-09-16,903.0
1180,2021-09-21,1000.0
1181,2021-09-23,903.0


In [147]:
# mask = daily_data['SCH_DEP_DATE'].dt.year > int(2016)
# slct_df=daily_data.where(mask)
# selected_yr_df=slct_df.dropna()
selected_yr_df=daily_data


In [148]:
def time_plot(data, x_col, y_col, title):
    fig, ax = plt.subplots(figsize=(18,4))
    sns.lineplot(x_col, y_col, data=data, ax=ax, color='mediumblue', label='Total Cargo')
#     print(data)
    second = data.groupby("SCH_DEP_DATE")[y_col].mean().reset_index()
    print(second)
    second.SCH_DEP_DATE = pd.to_datetime(second.SCH_DEP_DATE, format='%Y-%m-%d')
    sns.lineplot((second.SCH_DEP_DATE + datetime.timedelta(4*365/12)), y_col, data=second, ax=ax, color='red', label='Mean Cargo')   
    
    ax.set(xlabel = "Date",
           ylabel = "Cargo",
           title = title)
    
    sns.despine()
    
time_plot(selected_yr_df, 'SCH_DEP_DATE', 'CHG_WT_BU_sum', 'Daily Cargo Before Diff Transformation')

     SCH_DEP_DATE  CHG_WT_BU_sum
0      2018-01-03          171.1
1      2018-01-04          815.2
2      2018-01-05          551.7
3      2018-01-06         1820.2
4      2018-01-07        12000.3
...           ...            ...
1178   2021-09-14         1000.0
1179   2021-09-16          903.0
1180   2021-09-21         1000.0
1181   2021-09-23          903.0
1182   2021-09-28         1000.0

[1183 rows x 2 columns]


TypeError: tzinfo argument must be None or of a tzinfo subclass, not type 'UnitData'

In [ ]:
def get_diff(data):
    data=data.sort_values(by='SCH_DEP_DATE')
    data['cargo_diff'] = data.CHG_WT_BU_sum.diff()
    data = data.dropna()
    dir=os.path.dirname(os.path.abspath("__file__"))
    data.to_csv('../data/stationary_cargo_df.csv')
    return data
    
stationary_df = get_diff(selected_yr_df)
time_plot(stationary_df, 'SCH_DEP_DATE', 'CHG_WT_BU_sum', 'Daily Cargo After Diff Transformation')

In [ ]:
dopColm=['CHG_WT_BU_sum']

In [ ]:


def plots(data, lags=None):
    
    # Convert dataframe to datetimdopColm
    dt_data = data.set_index('SCH_DEP_DATE').drop(dopColm, axis=1)
    dt_data.info()
    dt_data.dropna(axis=0)
    layout = (1, 3)
    raw  = plt.subplot2grid(layout, (0, 0))
    acf  = plt.subplot2grid(layout, (0, 1))
    pacf = plt.subplot2grid(layout, (0, 2))
    
    dt_data.plot(ax=raw, figsize=(12, 5), color='mediumblue')
    smt.graphics.plot_acf(dt_data, lags=lags, ax=acf, color='mediumblue')
    smt.graphics.plot_pacf(dt_data, lags=lags, ax=pacf, color='mediumblue')
    sns.despine()
    plt.tight_layout()

plots(stationary_df, lags=18);
display(stationary_df)

# Preparing Dataset Modeling

In [ ]:
#create dataframe for transformation from time series to supervised
def generate_supervised(data):
    supervised_df = data.copy()
    
    #create column for each lag
    for i in range(1,19):
        col_name = 'lag_' + str(i)
        supervised_df[col_name] = supervised_df['cargo_diff'].shift(i)
    
    #drop null values
    supervised_df = supervised_df.dropna().reset_index(drop=True)
    dir=os.path.dirname(os.path.abspath("__file__"))
    supervised_df.to_csv(dir+'/data/model_df.csv', index=False)
    
    return supervised_df
# display(stationary_df)
model_df = generate_supervised(stationary_df)

model_df


# ARIMA Modeling

In [ ]:
def generate_arima_data(data):
    dt_data = data.set_index("SCH_DEP_DATE").drop('CHG_WT_BU_sum', axis=1)
    dt_data.dropna(axis=0)
    # dir=os.path.dirname(os.path.abspath("__file__"))
    # dt_data.to_csv(dir+'/data/arima_df.csv')
    writeToCsv(dt_data,"arima_df.csv")
    return dt_data
datetime_df = generate_arima_data(stationary_df)
datetime_df
